## Feedback
	
1.) 2/2

2.) 2/2

3.) 2/2

4.) 2/2

Great job!

# Homework 1
## Balázs Menkó (O67UT7)

Your answers should contain an SQL query, a short explanation, and the result of running the query. If you've worked in a Jupyter notebook, you can put all answers there; otherwise, it's preferred that the first two should be submitted as a text field, while the last one as a separate file.

In [1]:
# packages
import psycopg2
import psycopg2.extras
import pandas as pd

In [2]:
# Create connection
pgsql_settings = {
    'pguser' : 'menkobalazs1',
    'pgpasswd' : 'O67UT7',
    'pghost' : 'postgres-datasci.db-test',
    'pgport' : 5432,
    'pgdb' : 'menkobalazs1_homework',
    'schema' : 'public'
}

def connect_from_settings(settings):
    return psycopg2.connect(
        host = settings['pghost'],
        port = settings['pgport'],
        database = settings['pgdb'],
        user = settings['pguser'],
        password = settings['pgpasswd'],
        options=f'--search_path={settings["schema"]}'
    )

def run_query(query):
    "Run a query in one line"
    connection = connect_from_settings(pgsql_settings)
    cursor = connection.cursor(cursor_factory = psycopg2.extras.DictCursor)
    # Run the query
    cursor.execute(query)
    # Get the results
    dict_res = cursor.fetchall()
    # Since the results are in a list, we'll only ask for the keys() of the first element, 
    # and list the iterator returned by keys()
    df = pd.DataFrame(dict_res,columns=list(dict_res[0].keys())) 
    # Close everything
    cursor.close()
    connection.close()
    return df

# Create tables

```python
connection = connect_from_settings(pgsql_settings)
cursor = connection.cursor()
# Run the query
cursor.execute("""
    *create_a_table: copy the generator text and run the code*
""")
# COMMIT CHANGES
connection.commit()
# Close everything
cursor.close()
connection.close()
```

### Tables and columns:
- `branch`: location_id, location, manager, café, drive_thru, wi_fi_password
- `employee`: personel_id, first_name, last_name, date_of_birth, role, salary, supervisor_id, branch_id
- `purchases`: transaction_id, transaction_date, location_id, cashier_id, total_paid, items_sold
- `supplier`: branch_id, supplier_name, product
- `menu`: item_id, item_name, item_price, item_type

# Task 1
An alternative use for HAVING: Find the total number of employees for each location that has an average salary greater than 100 (thousand) dollars! (2 points)

In [3]:
query = """
    SELECT branch.location, COUNT(personel_id) AS total_num_of_employees
    FROM employee
    JOIN branch  ON branch.location_id = employee.branch_id 
    GROUP BY location
    HAVING AVG(salary) > 100
    ORDER BY location
"""
run_query(query)

,location,total_num_of_employees
0,Cape Town,7
1,London,6
2,Tokyo,4


#### Explanation:
Firstly, select 
 - the `locations` from `branch` table,  
 - count the number of `personel_id`, which will be the total number of employees
 
```sql
SELECT branch.location, COUNT(personel_id) AS total_num_of_employees
```
Use the `employee` table for searching.
```sql
FROM employee
```
To get the real name of locations join the `branch` table with `employee` table.
```sql
JOIN branch ON branch.location_id = employee.branch_id 
```
To count, we need to create a group by `location`.
```sql
GROUP BY location
```
We only need those whose salary is above 100.
```sql
HAVING AVG(salary) > 100
```
Finally, with the abc order, order by `location`.
```sql
ORDER BY location
```


# Task 2
Employees under 25 years old are eligible for tax relief. Create a query that lists each employee's full name in one column and whether or not they’re eligible for the discount in the second column. Order them alphabetically! (2 points)

In [4]:
query = """
    SELECT CONCAT(first_name, ' ', last_name) AS full_name, 
        CASE 
         WHEN (EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM date_of_birth)) < 25 
         THEN 'eligible'
         ELSE 'not eligible'
        END AS tax_relief_status
    FROM employee
    ORDER BY full_name
"""
run_query(query)

,full_name,tax_relief_status
0,Abby Archell,not eligible
1,Ambur Domengue,not eligible
2,Anthea Chesswas,not eligible
3,Belita Teale,not eligible
4,Benjamin Rowbotham,eligible
5,Casey Prydie,not eligible
6,Chiho Sasaki,eligible
7,Cristina Gue,not eligible
8,Ebeneser Neiland,not eligible
9,Evin Beine,not eligible


#### Explanation:
Firstly, select the concatenation of first and last name, and create an `if-else` statement to decide the age is greater or not then 25. Use `EXTRACT()` and `YEAR FROM` to get date. 
```sql
SELECT CONCAT(first_name, ' ', last_name) AS full_name, 
    CASE 
     WHEN (EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM date_of_birth)) < 25 
     THEN 'eligible'
     ELSE 'not eligible'
    END AS tax_relief_status
```
Use the `employee` table.
```sql
FROM employee
```
And finally order the list alphabetically.
```sql
ORDER BY full_name
```


# Task 3
Determine the total income of each branch from '2022-09-15' to '2022-09-22', but only show branches that had a higher total income than 850 in descending order! (2 points)

In [5]:
query = """
    SELECT location, SUM(total_paid) AS total_income
    FROM purchases
    JOIN branch  ON branch.location_id = purchases.location_id 
    WHERE transaction_date BETWEEN '2022-09-15' AND '2022-09-22'
    GROUP BY location
    HAVING SUM(total_paid) > 850
    ORDER BY total_income DESC
"""
run_query(query)

,location,total_income
0,New York,988.0
1,Tokyo,852.0


#### Explanation:
 Select the `location` and the total income (sum of `total_paid`) for each location.
```sql
SELECT location, SUM(total_paid) AS total_income
```
From the `purchases` table.
```sql
FROM purchases
```
Join the `branch` table to get the location names.
```sql
JOIN branch  ON branch.location_id = purchases.location_id 
```
Filter for transactions that occurred between September 15, 2022, and September 22, 2022
```sql
WHERE transaction_date BETWEEN '2022-09-15' AND '2022-09-22'
```
Group the results by `location` so that we can sum the `total_paid` for each location
```sql
GROUP BY location
```
Only include locations where the total income is greater than 850.
```sql
HAVING SUM(total_paid) > 850
```
Order the results by the total income in descending order.
```sql
ORDER BY total_income DESC
```

# Task 4
What was the longest order for each branch location? (2 points)

In [6]:
# First solution for only one maximum per location
query = """
    SELECT location, MAX(ARRAY_LENGTH(items_sold, 1)) AS num_of_max_sold_items
    FROM purchases
    JOIN branch ON branch.location_id = purchases.location_id
    GROUP BY location
"""
run_query(query)

,location,num_of_max_sold_items
0,Cape Town,4
1,New York,4
2,Tokyo,4
3,Wellington,4
4,London,4


In [7]:
# Second solution for all maximum per location with sold items.
query = """
    SELECT location, items_sold
    FROM purchases
    JOIN branch ON branch.location_id = purchases.location_id
    
    WHERE ARRAY_LENGTH(items_sold, 1) = (
        SELECT MAX(ARRAY_LENGTH(items_sold, 1)) 
        FROM purchases
    )
    ORDER BY location    
"""
run_query(query)

,location,items_sold
0,Cape Town,"[1005, 1006, 1002, 1004]"
1,Cape Town,"[1007, 1001, 1008, 1005]"
2,Cape Town,"[1008, 1007, 1007, 1004]"
3,Cape Town,"[1008, 1005, 1007, 1002]"
4,Cape Town,"[1007, 1005, 1008, 1001]"
...,...,...
244,Wellington,"[1004, 1005, 1007, 1004]"
245,Wellington,"[1002, 1001, 1003, 1002]"
246,Wellington,"[1006, 1002, 1007, 1002]"
247,Wellington,"[1002, 1001, 1007, 1004]"


#### Explanation:
Select the `location` and `items_sold` columns
```sql
SELECT location, items_sold
FROM purchases
```
Join the `purchases` table with the `branch` table on the `location_id` field
```sql
JOIN branch ON branch.location_id = purchases.location_id
```
Filter the results where the length of the `items_sold` array equals the maximum array length found in all rows of the `purchases` table.
```sql
WHERE ARRAY_LENGTH(items_sold, 1) = (
    --Subquery to find the maximum length of the `items_sold` array from the `purchases` table.--
    SELECT MAX(ARRAY_LENGTH(items_sold, 1)) 
    FROM purchases
)
```
Order the results by the `location` in ascending order.
```sql
ORDER BY location
```
